<h2 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> TPS Sep 2021 - EDA</h2> 


[![EDA-Image.png](https://i.postimg.cc/Nj1tLsHM/EDA-Image.png)](https://postimg.cc/v4HkNdXF)

<p style = "font-size:30px; color: #007580 ;background-color:  ; text-align: left; border-radius: 5px 5px; padding: 5px" ><strong> Here are my other notebooks, please have a look and definitely you will find it useful. Happy reading 🙂</strong></p>
<ol>
<li><a href ="https://www.kaggle.com/vinayakshanawad/industrial-safety-complete-solution">Industrial Safety - Complete Solution</a></li>
<li><a href ="https://www.kaggle.com/vinayakshanawad/eda-statistical-analysis-hypothesis-testing">EDA - Statistical Analysis - Hypothesis Testing</a></li>
<li><a href ="https://www.kaggle.com/vinayakshanawad/random-forest-with-bootstrap-sampling-for-beginner">Random Forest with Bootstrap Sampling for beginner</a></li>
<li><a href ="https://www.kaggle.com/vinayakshanawad/amazon-electronics-eda-recommender-system">Amazon Electronics - EDA - Recommender System</a></li>
<li><a href ="https://www.kaggle.com/vinayakshanawad/personal-loan-logistic-reg-accuracy-90-41">Personal Loan - Logistic Reg - Accuracy = 90.41%</a></li>
</ol>

<a id = '0'></a>
<h2 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #007580; color : #fed049; border-radius: 5px 5px; text-align:center; font-weight: bold" >Table of Contents</h2> 

1. [Overview](#1.0)
2. [Import the necessary libraries](#2.0)
3. [Data Collection](#3.0)
4. [Data Cleaning](#4.0)
	- [4.1 Check Duplicates](#4.1)
	- [4.2 Check Missing Values](#4.2)
    - [4.3 Handle Missing Values](#4.3)
	- [4.4 Check Outliers in Train Set](#4.4)
	- [4.5 Check Outliers in Test Set](#4.5)
	- [4.6 Data Cleaning Summary](#4.6)
5. [EDA (Data Analysis and Preparation)](#5.0)
	- [5.1 Univariate Analysis - Train set - Numerical columns](#5.1)
	- [5.2 Check Skewness](#5.2)
	- [5.3 Check Kurtosis](#5.3)
	- [5.4 Univariate analysis - Target (Categorical) column](#5.4)
	- [5.5 Univariate analysis - Test set - Numerical columns](#5.5)
    - [5.6 Study Summary Statistics](#5.6)
	- [5.7 Study Correlation](#5.7)
	- [5.8 EDA (Exploratory Data Analysis) Summary](#5.8)
6. [Future work](#5.0)

<a id = '1.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 1. Overview </h2> 

### Data Description:

For this competition, we will predict whether a customer made a claim upon an insurance policy. The ground truth claim is binary valued, but a prediction may be any number from 0.0 to 1.0, representing the probability of a claim. The features in this dataset have been anonymized and may contain missing values.

### Files

- train.csv - the training data with the target claim column
- test.csv - the test set; you will be predicting the claim for each row in this file
- sample_submission.csv - a sample submission file in the correct format

### Evaluation

Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

<a id = '2.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 2. Import the necessary libraries </h2> 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(color_codes = True)

# used to supress display of warnings
import warnings

import itertools

# pre-processing methods
from sklearn.preprocessing import StandardScaler

# evaluation methods
from sklearn.pipeline import Pipeline

#### Setting Options

In [ ]:
# suppress display of warnings
warnings.filterwarnings('ignore')

# display all dataframe columns
pd.options.display.max_columns = None

# to set the limit to 3 decimals
pd.options.display.float_format = '{:.7f}'.format

# display all dataframe rows
pd.options.display.max_rows = None

<a id = '3.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 3. Data Collection </h2> 

In [ ]:
INPUT = '../input/tabular-playground-series-sep-2021/'

train = pd.read_csv(INPUT + 'train.csv')
test = pd.read_csv(INPUT + 'test.csv')
submission = pd.read_csv(INPUT + 'sample_solution.csv')

In [ ]:
print('Number of rows = {0} and Number of columns = {1} in train set'.format(train.shape[0], train.shape[1]))
print('Number of rows = {0} and Number of columns = {1} in test set'.format(test.shape[0], test.shape[1]))
print('Number of rows = {0} and Number of columns = {1} in sample solution set'.format(submission.shape[0], submission.shape[1]))

In [ ]:
# Get train set - top 5 rows
train.head()

In [ ]:
# Get test set - top 5 rows
test.head()

In [ ]:
# Get submission set - top 5 rows
submission.head()

#### Data type of each attribute

In [ ]:
# Check train set info
train.info()

**Observations**

* Except id and claim columns, all columns are decimals (float)
* Id column is numerical column
* Claim column is categorical (binary) column

In [ ]:
# Reference - https://www.kaggle.com/maksymshkliarevskyi/tps-sep-all-for-start-eda-xgb-catboost-baseline
my_colors = ['#DC143C', '#FF1493', '#FF7F50', '#FFD700', '#32CD32', '#4ddbff', '#1E90FF', '#663399', '#708090']

dtypes = train.dtypes.value_counts().reset_index()

plt.figure(figsize = (12, 1))
plt.title('Data types\n')
plt.barh(str(dtypes.iloc[0, 0]), dtypes.iloc[0, 1], label = str(dtypes.iloc[0, 0]), color = my_colors[4])
plt.barh(str(dtypes.iloc[0, 0]), dtypes.iloc[1, 1], left = dtypes.iloc[0, 1], label = str(dtypes.iloc[1, 0]), color = my_colors[5])
plt.legend(loc = 'upper center', ncol = 3, fontsize = 13, bbox_to_anchor = (0.5, 1.45), frameon = False)
plt.yticks('')
plt.text(110, -0.9, '', size = 8)
plt.show()

**Observations**

* Except id and claim columns, all columns are decimals (float)
* Id column is numerical column

In [ ]:
# Check test set info
test.info()

<p style = "font-size:20px; color: #007580 "><strong> Data Collection Summary </strong></p>

1. There are about 957919 rows and 120 columns in train dataset.
2. There are about 493474 rows and 119 columns in test dataset.
3. We see there are total 118 float64 types and 2 int64 types variables present in train dataset.
4. We see there are total 118 float64 types and 1 int64 types variables present in test dataset.

* **Binary Categorical columns** - claim

* **Numerical columns** - From f1 to f118

<a id = '4.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 4. Data Cleaning </h2> 

In [ ]:
# Drop id column in train and test set
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)

<a id = '4.1'></a>
<p style = "font-size:20px; color: #007580 "><strong> 4.1 Check Duplicates </strong></p> 

In [ ]:
# Check duplicates in train set
train.duplicated().sum()

In [ ]:
# Check duplicates in test set
test.duplicated().sum()

<a id = '4.2'></a>
<p style = "font-size:20px; color: #007580 "><strong> 4.2 Check Missing Values </strong></p> 

In [ ]:
# Number of missing values in each column in train set
train.isnull().sum().sort_values(ascending = False).to_frame().rename({0:'Counts'}, axis = 1).T.style.background_gradient('crest')

In [ ]:
# Reference - https://www.kaggle.com/snikhil17/making-basic-eda-attractive

missing = (train.isnull().sum()).to_frame().reset_index().rename({0:'Counts'}, axis = 1)
plt.figure(figsize=(20, 5))
ax = sns.barplot(missing['index'], missing['Counts'], palette  = 'magma_r')
plt.title("Number of Missing Values in Train set", fontsize = 20)
plt.xticks(fontsize = 7, rotation = 90)
plt.xlabel("Variables")
plt.ylabel("Number of Missing Values")
plt.show();

In [ ]:
# % of missing values in each column in Train set
display(round((train.isnull().sum() / (len(train.index)) * 100) , 2).sort_values(ascending = False).to_frame().rename({0:'Percentage(%)'}, axis = 1).T.style.background_gradient('magma_r'))

In [ ]:
missing = (train.isnull().sum() / (len(train.index)) * 100).to_frame().reset_index().rename({0:'Percentage(%)'}, axis = 1)
plt.figure(figsize=(20, 5))
ax = sns.barplot(missing['index'], missing['Percentage(%)'], palette  = 'viridis')
plt.title("% of Missing Values in Train set", fontsize = 20)
plt.xticks(fontsize =7, rotation = 90)
plt.xlabel("Variables")
plt.ylabel("% of Missing Values")
plt.show();

In [ ]:
# Number of missing values in each column in test set
test.isnull().sum().sort_values(ascending = False).to_frame().rename({0:'Counts'}, axis = 1).T.style.background_gradient('crest')

In [ ]:
missing = (test.isnull().sum()).to_frame().reset_index().rename({0:'Counts'}, axis = 1)
plt.figure(figsize=(20, 5))
ax = sns.barplot(missing['index'], missing['Counts'], palette  = 'ch:start=.2,rot=-.3')
plt.title("Number of Missing Values in Test set", fontsize = 20)
plt.xticks(fontsize = 7, rotation = 90)
plt.xlabel("Variables")
plt.ylabel("Number of Missing Values")
plt.show();

In [ ]:
# % of missing values in each column in Test set
display(round((test.isnull().sum() / (len(test.index)) * 100) , 2).sort_values(ascending = False).to_frame().rename({0:'Percentage(%)'}, axis = 1).T.style.background_gradient('magma_r'))

In [ ]:
missing = (train.isnull().sum() / (len(train.index)) * 100).to_frame().reset_index().rename({0:'Percentage(%)'}, axis = 1)
plt.figure(figsize=(20, 5))
ax = sns.barplot(missing['index'], missing['Percentage(%)'], palette  = 'flare')
plt.title("% of Missing Values in Test set", fontsize = 20)
plt.xticks(fontsize =7, rotation = 90)
plt.xlabel("Variables")
plt.ylabel("% of Missing Values")
plt.show();

<a id = '4.3'></a>
<p style = "font-size:20px; color: #007580 "><strong> 4.3 Handle Missing Values </strong></p> 

In [ ]:
# Replace missing missing numerical values with it's mean value in train set
for i in train.columns[(train.columns != 'id') & (train.columns != 'claim')]:
    train[i].fillna(train[i].mean(), inplace = True)

In [ ]:
# Number of missing values in each column in train set
train.isnull().sum().sort_values(ascending = False).to_frame().rename({0:'Counts'}, axis = 1).T.style.background_gradient('crest')

In [ ]:
# Replace missing missing numerical values with it's mean value in test set
for i in test.columns[(test.columns != 'id')]:
    test[i].fillna(test[i].mean(), inplace = True)

In [ ]:
# Number of missing values in each column in test set
test.isnull().sum().sort_values(ascending = False).to_frame().rename({0:'Counts'}, axis = 1).T.style.background_gradient('crest')

<a id = '4.4'></a>
<p style = "font-size:20px; color: #007580 "><strong> 4.4 Check Outliers in Train Set </strong></p> 

In [ ]:
# Reference: https://www.kaggle.com/suharkov/sep-2021-playground-eda-no-model-for-now
df_plot = ((train - train.min())/(train.max() - train.min()))

fig, ax = plt.subplots(4, 1, figsize = (25,25))
sns.boxplot(data = df_plot.iloc[:, 1:30], ax = ax[0])
sns.boxplot(data = df_plot.iloc[:, 30:60], ax = ax[1])
sns.boxplot(data = df_plot.iloc[:, 60:90], ax = ax[2])
sns.boxplot(data = df_plot.iloc[:, 90:120], ax = ax[3])

**Observations**

* Except these features - f5, f7, f12, f25, f28, f29, f34, f37, f40, f42, f50, f56, f57, f60, f61, f65, f67, f70, f72, f75, f81, f88, f93, f97, f100, f105, f109, f117 all other features are having outliers in train set.

<a id = '4.5'></a>
<p style = "font-size:20px; color: #007580 "><strong> 4.5 Check Outliers in Test Set </strong></p> 

In [ ]:
# Reference: https://www.kaggle.com/suharkov/sep-2021-playground-eda-no-model-for-now
df_plot = ((test - test.min())/(test.max() - test.min()))

fig, ax = plt.subplots(4, 1, figsize = (25,25))
sns.boxplot(data = df_plot.iloc[:, 1:30], ax = ax[0])
sns.boxplot(data = df_plot.iloc[:, 30:60], ax = ax[1])
sns.boxplot(data = df_plot.iloc[:, 60:90], ax = ax[2])
sns.boxplot(data = df_plot.iloc[:, 90:120], ax = ax[3])

**Observations**

* Except these features - f5, f7, f12, f25, f28, f29, f34, f37, f40, f42, f50, f56, f57, f60, f61, f65, f67, f70, f72, f75, f81, f88, f93, f97, f100, f105, f109, f117 all other features are having outliers in test set.

<a id = '4.6'></a>
<p style = "font-size:20px; color: #007580 "><strong> 4.6 Data Cleaning Summary </strong></p>

1. No duplicate instances in both train and test set.
2. % of missing values in both train and test set is 1.57% to 1.63%.
3. Replaced missing missing numerical values with it's mean value in both train set and test set..
3. Except these features - f5, f7, f12, f25, f28, f29, f34, f37, f40, f42, f50, f56, f57, f60, f61, f65, f67, f70, f72, f75, f81, f88, f93, f97, f100, f105, f109, f117 all other features are having outliers in train set.
4. Except these features - f5, f7, f12, f25, f28, f29, f34, f37, f40, f42, f50, f56, f57, f60, f61, f65, f67, f70, f72, f75, f81, f88, f93, f97, f100, f105, f109, f117 all other features are having outliers in test set.

    **Note: Need to handle Outliers in both Train and Test set.**

<a id = '5.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 5. EDA (Data Analysis and Preparation) </h2> 

<a id = '5.1'></a>
<p style = "font-size:20px; color: #007580 "><strong> 5.1 Univariate Analysis - Train set - Numerical columns</strong></p> 

In [ ]:
num_cols = train.drop('claim', axis = 1).columns

In [ ]:
cols = num_cols[0:110]
length = len(cols)
cs = ["b","r","g","c","m","k","lime","c","b","r"]*11
fig = plt.figure(figsize=(20, 100))

for i,j,k in itertools.zip_longest(cols, range(length),cs):
    plt.subplot(22,5,j+1)
    fig.tight_layout(pad=2.0)
    ax = sns.kdeplot(train[i], color=k)
    ax.set_facecolor("w")
    plt.axvline(train[i].mean(), linestyle="dashed", label="mean", color="k")
    plt.legend(loc="best")
    plt.title(i,color="navy")
    plt.xlabel("")

**Observations**

* **f34, f57 and f97 features - Normally distributed.**
* **f2, f31, f58, f60, f91, f94, f110 - Left skewed distribution - all these features are skewed to lower values.**
* **f1, f17, f22, f46, f79 - seems to normally distributed.**
* **f23, f55, f66   - Moderately left skewed distribution.**
* **f36, f48, f69, f90, f99  - Moderately right skewed distribution.**
* **Except all above features, rest of the features are skewed to higher values.**

In [ ]:
cols = num_cols[110:119]
length = len(cols)
cs = ["b","r","g","c","m","k","lime","c"]
fig = plt.figure(figsize=(20, 8))

for i,j,k in itertools.zip_longest(cols, range(length),cs):
    plt.subplot(2,5,j+1)
    fig.tight_layout(pad=2.0)
    ax = sns.kdeplot(train[i], color=k)
    ax.set_facecolor("w")
    plt.axvline(train[i].mean(), linestyle="dashed", label="mean", color="k")
    plt.legend(loc="best")
    plt.title(i,color="navy")
    plt.xlabel("")

**Observations**

* **f113 feature - Normally distributed.**
* **Except f113 feature, rest of the features are skewed to higher values.**

<a id = '5.2'></a>
<p style = "font-size:20px; color: #007580 "><strong> 5.2 Check Skewness </strong></p>

In [ ]:
train[num_cols].skew().sort_values(ascending = False).to_frame().rename({0:'Skewness'}, axis = 1).T.style.background_gradient('crest')

**Observations**

* **f34, f57, f97, f2, f31, f58, f60, f91, f94, f110, f1, f17, f22, f46, f79, f23, f55, f66, f36, f48, f69, f90, f99 - Except these features, rest of the features are skewed to higher values.**
* **f1, f65, f60, f11, f50, f31, f55, f5, f24, f66, f94, f23, f2, f13, f110, f58, f91, f46 - These features are skewed to lower values.**

<a id = '5.3'></a>
<p style = "font-size:20px; color: #007580 "><strong> 5.3 Check Kurtosis </strong></p>

In [ ]:
train[num_cols].kurtosis().sort_values(ascending = False).to_frame().rename({0:'Kurtosis'}, axis = 1).T.style.background_gradient('crest_r')

**Observations**

* **f10, f22, f46, f104, f17, f92, f96, f43, f76, f53, f3, f6, f36, f69, f32, f51, f21, f116, f114, f26, f115, f73 - These features are having more peaked distribution which is Leptokurtic.**
* **f62, f33, f112, f4, f64, f83, f118, f71, f15, f18, f106, f113, f44, f38, f30, f63, f11, f82, f79, f48, f52, f74, f103, f68, f39, f87, f110, f1, f91, f89, f90, f13, f14, f99, f102, f8, f101, f80 - These features are having less peaked distribution which is Platykurtic.**

<a id = '5.4'></a>
<p style = "font-size:20px; color: #007580 "><strong> 5.4 Univariate analysis - Target (Categorical) column </strong></p>

In [ ]:
plt.figure(figsize = (10,6))
train['claim'].value_counts().plot.pie(autopct = '%.1f', colors = ['powderblue', 'slateblue'])
plt.title("Claim vlaue distribution", pad = 20, fontdict = {'size' : 15, 'color' : 'brown', 'weight' : 'bold'})
plt.show()

* **Above plot shows, claim distribution is almost equal.**

### Calculate claim ratio from target variable

In [ ]:
claim_yes = len(train.loc[train['claim'] == 1])
claim_no = len(train.loc[train['claim'] == 0])

print("Number of claimed insurance policies: {0} ({1:2.2f}%)".format(claim_yes, (claim_yes / (claim_yes + claim_no)) * 100 ))
print("Number of disclaimed insurance policies: {0} ({1:2.2f}%)".format(claim_no, (claim_no / (claim_yes + claim_no)) * 100))

* **So we have 49.85% customers claimed the insurance policies and 50.15% customers did not claim. And data is balanced as well.**

<a id = '5.5'></a>
<p style = "font-size:20px; color: #007580 "><strong> 5.5 Univariate analysis - Test set - Numerical columns </strong></p>

In [ ]:
cols = num_cols[0:110]
length = len(cols)
cs = ["b","r","g","c","m","k","lime","c","b","r"]*11
fig = plt.figure(figsize=(20, 100))

for i,j,k in itertools.zip_longest(cols, range(length),cs):
    plt.subplot(22,5,j+1)
    fig.tight_layout(pad=2.0)
    ax = sns.kdeplot(test[i], color=k)
    ax.set_facecolor("w")
    plt.axvline(train[i].mean(), linestyle="dashed", label="mean", color="k")
    plt.legend(loc="best")
    plt.title(i,color="navy")
    plt.xlabel("")

**Observations**

* **f34, f57 and f97 features - Normally distributed.**
* **f2, f31, f58, f60, f91, f94, f110 - Left skewed distribution - all these features are skewed to lower values.**
* **f1, f17, f22, f46, f79 - seems to normally distributed.**
* **f23, f55, f66   - Moderately left skewed distribution.**
* **f36, f48, f69, f90, f99  - Moderately right skewed distribution.**
* **Except all above features, rest of the features are skewed to higher values.**

In [ ]:
cols = num_cols[110:119]
length = len(cols)
cs = ["b","r","g","c","m","k","lime","c"]
fig = plt.figure(figsize=(20, 8))

for i,j,k in itertools.zip_longest(cols, range(length),cs):
    plt.subplot(2,5,j+1)
    fig.tight_layout(pad=2.0)
    ax = sns.kdeplot(test[i], color=k)
    ax.set_facecolor("w")
    plt.axvline(train[i].mean(), linestyle="dashed", label="mean", color="k")
    plt.legend(loc="best")
    plt.title(i,color="navy")
    plt.xlabel("")

**Observations**

* **f113 feature - Normally distributed.**
* **Except f113 features, rest of the features are skewed to higher values.**

**Note: Train and Test set distribution looks similar, that means they come from same population distribution.**

<a id = '5.6'></a>
<p style = "font-size:20px; color: #007580 "><strong> 5.6 Study Summary Statistics </strong></p>

In [ ]:
# Summary statistics for Train set
train.describe().style.background_gradient(cmap='magma_r')

**Observations**

* **f3, f8, f9, f10, f12, f21, f26, f27, f28, f32, f33, f35, f62, f65, f73, f74, f77, f78, f82, f84, f86, f92, f96, f98, f102, f103, f104, f108, f114, f116 features are having very high values compared to other features.**

In [ ]:
# Summary statistics for Test set
test.describe().style.background_gradient(cmap='inferno_r')

**Observations**

* **f3, f8, f9, f10, f12, f21, f26, f27, f28, f32, f33, f35, f62, f65, f73, f74, f77, f78, f82, f84, f86, f92, f96, f98, f102, f103, f104, f108, f114, f116 features are having very high values compared to other features.**

<a id = '5.7'></a>
<p style = "font-size:20px; color: #007580 "><strong> 5.7 Study Correlation </strong></p>

<p style = "font-size:20px; color: #007580 "><strong> Study Correlation in Train Set </strong></p>

In [ ]:
# Create a copy of Train set
train_corr = train.copy()

# Transform Train set independent features
scaler_X = StandardScaler()
pipeline = Pipeline(steps=[('s', scaler_X)])

train_corr[num_cols] = pipeline.fit_transform(train_corr[num_cols])

#### Looking at the train set, some of the features have very high values while other very low values, so let's scale/normalize the features to check the correlation between all features.

In [ ]:
corr = abs(train_corr.corr()) # correlation matrix
lower_triangle = np.tril(corr, k = -1)  # select only the lower triangle of the correlation matrix
mask = lower_triangle == 0  # to mask the upper triangle in the following heatmap

plt.figure(figsize = (140, 40))
sns.heatmap(lower_triangle, cmap = 'coolwarm', annot= True, xticklabels = corr.index, yticklabels = corr.columns, cbar= True, linewidths= 1, 
            mask = mask)
plt.show()

**Observations**

* **All independent featuers are having negligible correlation with other features.**
* **Even target feature - claim is having negligible correlation with all independent features.**

In [ ]:
train_corr.corr()[['claim']].T.style.background_gradient('magma_r')

In [ ]:
# Delete train_corr
del train_corr

<p style = "font-size:20px; color: #007580 "><strong> Study Correlation in Test Set </strong></p>

#### Looking at the Test set, some of the features have very high values while other very low values, so let's scale/normalize the features to check the correlation between all features.

In [ ]:
# Create a copy of Train set
test_corr = test.copy()

# Transform Test set independent features
test_corr[num_cols] = pipeline.fit_transform(test_corr[num_cols])

In [ ]:
corr = abs(test_corr.corr()) # correlation matrix
lower_triangle = np.tril(corr, k = -1)  # select only the lower triangle of the correlation matrix
mask = lower_triangle == 0  # to mask the upper triangle in the following heatmap

plt.figure(figsize = (140, 40))
sns.heatmap(lower_triangle, cmap = 'coolwarm', annot= True, xticklabels = corr.index, yticklabels = corr.columns, cbar= True, linewidths= 1, 
            mask = mask)
plt.show()

**Observations**

* **All independent featuers are having negligible correlation with other features.**

In [ ]:
# Delete test_corr
del test_corr

<a id = '5.8'></a>
<p style = "font-size:20px; color: #007580 "><strong> 5.8 EDA (Exploratory Data Analysis) Summary </strong></p>

**1. Univariate analysis in both Train and Test set.**
* f34, f57 and f97 features - Normally distributed.
* f2, f31, f58, f60, f91, f94, f110 - Left skewed distribution - all these features are skewed to lower values.
* f1, f17, f22, f46, f79 - seems to normally distributed.
* f23, f55, f66   - Moderately left skewed distribution.
* f36, f48, f69, f90, f99  - Moderately right skewed distribution.
* Except all above features, rest of the features are skewed to higher values.
* f113 feature - Normally distributed.
* Except f113 features, rest of the features are skewed to higher values.

	**Note: Train and Test set distribution looks similar, that means they come from same population distribution.**

**2. We have 49.85% customers claimed the insurance policies and 50.15% customers did not claim. And data is balanced as well.**
**3. Skewness details:**
* f34, f57, f97, f2, f31, f58, f60, f91, f94, f110, f1, f17, f22, f46, f79, f23, f55, f66, f36, f48, f69, f90, f99 - Except these features, rest of the features are skewed to higher values.
* f1, f65, f60, f11, f50, f31, f55, f5, f24, f66, f94, f23, f2, f13, f110, f58, f91, f46 - These features are skewed to lower values.
**4. Kurtosis details:**
* f10, f22, f46, f104, f17, f92, f96, f43, f76, f53, f3, f6, f36, f69, f32, f51, f21, f116, f114, f26, f115, f73 - These features are having more peaked distribution which is Leptokurtic.
* f62, f33, f112, f4, f64, f83, f118, f71, f15, f18, f106, f113, f44, f38, f30, f63, f11, f82, f79, f48, f52, f74, f103, f68, f39, f87, f110, f1, f91, f89, f90, f13, f14, f99, f102, f8, f101, f80 - These features are having less peaked distribution which is Platykurtic.

**5. Summary statistics:**
* f3, f8, f9, f10, f12, f21, f26, f27, f28, f32, f33, f35, f62, f65, f73, f74, f77, f78, f82, f84, f86, f92, f96, f98, f102, f103, f104, f108, f114, f116 features are having very high values compared to other features.

**6. Correlation analysis:**
* All independent featuers are having negligible correlation with other features.
* Even target feature - claim is having negligible correlation with all independent features.

<a id = '6.0'></a>
<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color: #007580; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> 6. Future work </h2> 

1. Data preprocessing.
2. Feature engineering.
3. Model building.
4. Hyper parameter tuning.
5. Final submission.

<p style = "font-size:30px; color: #007580 ;background-color:  ; text-align: left; border-radius: 5px 5px; padding: 5px" ><strong> Finally, please let me know your suggestions. Thanks for reading 🙂</strong></p>